<a href="https://colab.research.google.com/github/ArkS0001/Tacotron2/blob/main/AudFakeX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install PortAudio library
!apt-get install -y portaudio19-dev

# Install PyAudio
!pip install pyaudio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (110 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
import pyaudio
import numpy as np

def record_audio(duration=5, sample_rate=22050):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)

    frames = []
    for _ in range(0, int(sample_rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(np.frombuffer(data, dtype=np.int16))

    stream.stop_stream()
    stream.close()
    p.terminate()

    audio_data = np.hstack(frames)
    return audio_data, sample_rate

audio_data, sample_rate = record_audio()


OSError: [Errno -9996] Invalid input device (no default output device)

In [ ]:
import librosa
import pyworld as pw

def extract_features(audio_data, sample_rate):
    y = audio_data.astype(np.float32)

    # Extract mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y, sr=sample_rate, n_fft=1024, hop_length=256, n_mels=80)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Extract pitch
    _f0, t = pw.dio(y, sample_rate)
    f0 = pw.stonemask(y, _f0, t, sample_rate)

    return mel_spectrogram, f0

mel_spectrogram, f0 = extract_features(audio_data, sample_rate)


ModuleNotFoundError: No module named 'pyworld'

In [ ]:
import torch
from tacotron2.model import Tacotron2
from waveglow.denoiser import Denoiser

# Load the models
tacotron2 = Tacotron2()
tacotron2.load_state_dict(torch.load('tacotron2_statedict.pt'))
tacotron2.eval().cuda()

waveglow = torch.load('waveglow_statedict.pt')['model']
waveglow.eval().cuda()
denoiser = Denoiser(waveglow).cuda()


ModuleNotFoundError: No module named 'tacotron2'

In [ ]:
# Step 1: Clone the Tacotron2 and WaveGlow repositories
!git clone https://github.com/NVIDIA/tacotron2.git
!git clone https://github.com/NVIDIA/waveglow.git

# Step 2: Install dependencies
!pip install -r tacotron2/requirements.txt
!pip install -r waveglow/requirements.txt

# Step 3: Download pretrained models
# For example, using gdown to download from Google Drive
!gdown --id '1RPehARwQU7VNI6Wk3F5s4pZQ2BPXN3_S' -O tacotron2_statedict.pt
!gdown --id '1W9aX39WAz5QGlVMejG7u1QAsUXc--kZe' -O waveglow_256channels_universal_v5.pt

# Step 4: Set up environment and paths
import sys
sys.path.append('tacotron2')
sys.path.append('waveglow')

import torch
from tacotron2.model import Tacotron2
from tacotron2.text import text_to_sequence
from waveglow.denoiser import Denoiser

# Load Tacotron2 model
tacotron2 = Tacotron2()
tacotron2.load_state_dict(torch.load('tacotron2_statedict.pt'))
tacotron2.cuda().eval().half()

# Load WaveGlow model
waveglow = torch.load('waveglow_256channels_universal_v5.pt')['model']
waveglow.cuda().eval().half()
for m in waveglow.modules():
    if 'Conv' in str(type(m)):
        setattr(m, 'padding_mode', 'zeros')
waveglow_denoiser = Denoiser(waveglow)

# Function to convert text to speech
def text_to_speech(text):
    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()

    # Tacotron2 Inference
    with torch.no_grad():
        mel_outputs, mel_outputs_postnet, _, alignments = tacotron2.inference(sequence)

    # WaveGlow Inference
    with torch.no_grad():
        audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)

    # Denoiser
    audio_denoised = waveglow_denoiser(audio, strength=0.01)[:, 0]
    return audio_denoised

# Example usage
text = "Hello, how are you?"
audio = text_to_speech(text)

# Save the audio to a file
from scipy.io.wavfile import write
audio_numpy = audio.cpu().numpy().astype(np.float32)
write("output.wav", 22050, audio_numpy)


Cloning into 'tacotron2'...
remote: Enumerating objects: 412, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 412 (delta 2), reused 3 (delta 1), pack-reused 406
Receiving objects: 100% (412/412), 2.70 MiB | 2.00 MiB/s, done.
Resolving deltas: 100% (202/202), done.
Cloning into 'waveglow'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 196 (delta 2), reused 2 (delta 0), pack-reused 190
Receiving objects: 100% (196/196), 437.57 KiB | 24.31 MiB/s, done.
Resolving deltas: 100% (108/108), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 32.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.2 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0

In [ ]:
def synthesize_voice(text, mel_spectrogram, pitch, tacotron2, waveglow, denoiser):
    sequence = np.array([text_to_sequence(text, ['english_cleaners'])])[None, :]
    sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.long)

    mel_spectrogram = torch.tensor(mel_spectrogram).unsqueeze(0).to(device='cuda', dtype=torch.float32)
    pitch = torch.tensor(pitch).unsqueeze(0).to(device='cuda', dtype=torch.float32)

    with torch.no_grad():
        mel_outputs, mel_outputs_postnet, _, alignments = tacotron2.inference(sequence)
        audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
        audio = denoiser(audio, strength=0.01)[:, 0]

    return audio.cpu().numpy()

# Example text for synthesis
text = "Hello, how are you?"
cloned_audio = synthesize_voice(text, mel_spectrogram, f0, tacotron2, waveglow, denoiser)

# Save to file
import soundfile as sf
sf.write('cloned_output.wav', cloned_audio, sample_rate)
